# DatasetFolder & ImageFolder

- 해당 내용은 [Pytorch Github 코드](https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py)를 참조함

In [1]:
class Config():
    training_dir = "./data/faces/training/"
    testing_dir = "./data/faces/testing/"
    train_batch_size = 64
    train_number_epochs = 100

# [DatasetFolder](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.DatasetFolder)

- 데이터 샘플이 아래와 같이 정렬되는 일반 데이터 로더

```bash
root/class_x/xxx.ext
root/class_x/xxy.ext
root/class_x/xxz.ext

root/class_y/123.ext
root/class_y/nsdf3.ext
root/class_y/asd932_.ext
```

**Parameters**

- **root** *(string)* – Root directory path.
- **loader** *(callable)* – A function to load a sample given its path. -> 데이터 로더를 이야기한다.
- **extensions** *(list[string])* – A list of allowed extensions. -> 폴더 내부의 데이터들 중에 허용할 extension리스트
- **transform** *(callable, optional)* – A function/transform that takes in a sample and returns a transformed version. E.g, transforms.RandomCrop for images.

<br/>

**`__getitem__(index)`**
- **Parameters**:	index (int) – Index
- **Returns**:	(sample, target) where target is class_index of the target class.
- **Return type**:	tuple

In [7]:
import torch
import torchvision
import torchvision.datasets as dset

In [8]:
folder_dataset = dset.DatasetFolder(root=Config.training_dir, loader=default_loader, extensions=IMG_EXTENSIONS)

print(folder_dataset)
print(folder_dataset.__getitem__(0))
print(folder_dataset.__getitem__(1))
print(folder_dataset.__getitem__(3))

Dataset DatasetFolder
    Number of datapoints: 370
    Root Location: ./data/faces/training/
    Transforms (if any): None
    Target Transforms (if any): None
(<PIL.Image.Image image mode=RGB size=92x112 at 0x11E88DB70>, 0)
(<PIL.Image.Image image mode=RGB size=92x112 at 0x11E877E80>, 0)
(<PIL.Image.Image image mode=RGB size=92x112 at 0x11E877AC8>, 0)


## More detail

### 1. DatasetFolder class

- 시퀸스 객체이다.
- 우리가 이전에 확인한, `torch.utils.data.Dataset`과 같은 방식으로 구성되어있다.

<br/>

**`__init__`**

1. 클래스를 찾는다.
2. dataset을 만든다.
3. 받은 파라미터를 객체 멤버로 할당한다.

<br/>

**`__getitem__`**
1. sample 리스트에서 특정 index의 샘플을 가져온다.
    - 해당 sample은 (path, target)으로 구성된 튜플이다.
2. loader에 path를 전달하여, 실질적인 sample을 가져온다.
    - 예를들어, image의 경로를 pillow Image loader에 전달해서, PIL Image object를 반환한다.
    - 이는 네트워크의 Input값을 torch.tensor로 변환하기 위한 전처리 작업이다.
3. transform 인자가 `!= None` 이면 Input과 target를 transform(data augmentation etc.)처리를 해준다.

<br/>

**`__len__`**
1. 이는 sample의 리스트의 전체 길이를 확인해서 반환한다.

In [ ]:
class DatasetFolder(torch.utils.data.Dataset):

    def __init__(self, root, loader, extensions, transform=None, target_transform=None):
        classes, class_to_idx = self._find_classes(root)
        samples = make_dataset(root, class_to_idx, extensions)
        if len(samples) == 0:
            raise(RuntimeError("Found 0 files in subfolders of: " + root + "\n"
                               "Supported extensions are: " + ",".join(extensions)))

        self.root = root
        self.loader = loader
        self.extensions = extensions

        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples
        self.targets = [s[1] for s in samples]

        self.transform = transform
        self.target_transform = target_transform

    def _find_classes(self, dir):
        """
        Finds the class folders in a dataset.
        Args:
            dir (string): Root directory path.
        Returns:
            tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
        Ensures:
            No class is a subdirectory of another.
        """
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

    def __len__(self):
        return len(self.samples)

    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str

## Deep dive inside to DatasetFolder 

### `__init__`

```python
def __init__(self, root, loader, extensions, transform=None, target_transform=None):
        # 클래스를 찾는다.
        classes, class_to_idx = self._find_classes(root)
        # 데이터셋을 만든다.
        samples = make_dataset(root, class_to_idx, extensions)
        # 데이터셋을 만들었는데, sample의 길이가 0이면, error를 발생한다.
        if len(samples) == 0:
            raise(RuntimeError("Found 0 files in subfolders of: " + root + "\n"
                               "Supported extensions are: " + ",".join(extensions)))

        # 기본적으로 받은 파라미터와, 생성하는 과정에서 만들어진 변수를 객체 멤버에 할당한다.
        self.root = root
        self.loader = loader
        self.extensions = extensions

        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples
        self.targets = [s[1] for s in samples]

        self.transform = transform
        self.target_transform = target_transform
```

<br/>

```python
def _find_classes(self, dir):
        """
        Finds the class folders in a dataset.
        Args:
            dir (string): Root directory path.
        Returns:
            tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
        Ensures:
            No class is a subdirectory of another.
            각 클래서는 독립적인 범주여아한다, 어디에 속해있고 이러면 안된다.
        """
        
        # 여기서의 classes는 각 폴더의 이름을 의미한다.
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else: 
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        
        # 얻은 폴더들의 이름을 class명이라고 생각하며, 이를 sorting한다.
        classes.sort()
        
        # 각 class 명과, class 명의 index(target value)를 dictionary 형태로 묶는다.
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx
```

- python3.5 이상에서 제공하는 개선된 directory iterator인 `os.scandir()`을 통해서 데이터를 파싱한다.
    - [PEP 471-- os.scandir() function -- a better and faster directory iterator](https://www.python.org/dev/peps/pep-0471/)

<br/>

**`os.scandir()`**

- `os.scandir()`은 기존의 `os.walk()`의 속도보다 2~20배(파일 시스템에 의존한다) 이상 빨라진 기능을 제공한다. `os.scandir()`은 대부분의 경우에서 `os.stat()` 호출을 피한다. 
    - This new function adds useful functionality and increases the speed of os.walk() by 2-20 times (depending on the platform and file system) by avoiding calls to os.stat() in most cases.

<br/>

- python의 build-in function인 `os.walk()`함수는 필요이상으로 느린데, 그 이유는 각 디렉토리마다 `os.listdir()`을 호출하기 때문이다. 
    - `os.listdir()`은 해당 항목이 디렉토리인지 파일인지 확인하기 위해서 `stat()` system call이나 `GetFileAttributes()`함수를 호출한다. (이는 폴더계층이 복잡할 수록 recursive하게 작동하여, 문제가 생기는 듯 하다.)
    
<br/>
    
- 하지만, Windows의 `-- FindFirstFile / FindNextFile`과, POSIX system의 `readdir`은 이미 반환된 파일이 디렉토리인지 파일인지 알려준다. 따라서 이러한 추가적인 함수 호출은 불필요하다.

<br/>
<br/>

```python
def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str
```

- `__repr__`은 공식적인(official) 문자열을 출력할때 사용한다. 반대로 `__str__`은 비공식적인(informal) 문자열을 출력할때 사용한다.
- 즉, `__str__` 은 사용자가 보기 쉬운 형태로 보여줄 때 사용하는 것이고, `__repr__` 은 시스템(python interpreter)이 해당 객체를 인식할 수 있는 공식적인 문자열로 나타내 줄 때 사용하는 것이다.
- `__repr__` 이 구현되어있고, `__str__` 이 구현되어있지 않은 경우에는 repr 과 str 은 동일한 결과를 나타낸다

- [[Python] str() 과 repr() 의 차이](https://pinocc.tistory.com/168)

<br/>

```python
print(repr(forlder_dataset))

>'Dataset DatasetFolder\n    Number of datapoints: 370\n    Root Location: ./data/faces/training/\n    Transforms (if any): None\n    Target Transforms (if any): None'
```


## Deep dive inside to Others

```python
IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', 'webp']


def has_file_allowed_extension(filename, extensions):
    """Checks if a file is an allowed extension.
    Args:
        filename (string): path to a file
        extensions (iterable of strings): extensions to consider (lowercase)
    Returns:
        bool: True if the filename ends with one of given extensions
    """
    
    # filename을 모두 소문자로 만든다음에, extension list에 있는지 확인한다.
    # any라는 build-in function을 통해서 확인하는구나..
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in extensions)

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    # 기존의 PIL.open() 함수는 image metafile만 읽고, 그 후에 load()함수를 통해서 pixel access를 진행함
    # 이로인해서 metafile을 못읽게 되면, 직접 ojbect를 파기하지 않는 이상, object가 살아있었음.
    # 이로인해 ResourceWarning이 발생하게 됨.
    
    # 추측컨데, 이를 수정하는 과정->open()과 load()를 병합하는 과정에서 PIL의 C바인딩과 PIL python 코드 매칭이 안된듯함
    # 따라서 C바인딩에서는 return value가 삭제되었지만, python에서는 진입하는 함수가 작동하여 자동으로 file을 close하지않고
    # 파기했기에 ResourceWarning 메시지가 떴음
    # 이를 피하기 위해서 아래와 같은 방식을 적용함
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

# image_backend()를 확인한 후에, 그에 맞춰서 default loader를 사용함
def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

def make_dataset(dir, class_to_idx, extensions):
    images = []
    
    # 상대경로를 절대경로로 변경
    dir = os.path.expanduser(dir)
    
    # 이름순으로 정렬
    for target in sorted(class_to_idx.keys()):
        
        # target data를 위한 path or dir이 있는지 확인
        d = os.path.join(dir, target)
        
        # d가 디렉토리가 아니면, 건너뛴다.
        if not os.path.isdir(d):
            continue

        # 얻은 target 데이터를 image list에 append함
        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                
                # 허용하는 extension인지 확인
                if has_file_allowed_extension(fname, extensions):
                    # path와 filename을 이용해서 최종 path 결정
                    path = os.path.join(root, fname)
                    
                    # 튜플로 path과 label index의 구성을 만듬
                    item = (path, class_to_idx[target])
                    
                    # sample list를 만듬
                    images.append(item)

    return images
```

# [ImageFolder](https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder)


- 이미지가 아래와 같이 정렬되는 일반 데이터 로더

```bash
root/dog/xxx.png
root/dog/xxy.png
root/dog/xxz.png

root/cat/123.png
root/cat/nsdf3.png
root/cat/asd932_.png
```

**Parameters**

- **root** *(string)* – Root directory path.
- **loader** *(callable)* – A function to load a sample given its path. -> 데이터 로더를 이야기한다.
- **extensions** *(list[string])* – A list of allowed extensions. -> 폴더 내부의 데이터들 중에 허용할 extension리스트
- **transform** *(callable, optional)* – A function/transform that takes in a sample and returns a transformed version. E.g, transforms.RandomCrop for images.

<br/>

**`__getitem__(index)`**
- **Parameters**:	index (int) – Index
- **Returns**:	(sample, target) where target is class_index of the target class.
- **Return type**:	tuple

In [20]:
class ImageFolder(DatasetFolder):

    def __init__(self, root, transform=None, target_transform=None,
                 loader=default_loader):
        super(ImageFolder, self).__init__(root, loader, IMG_EXTENSIONS,
                                          transform=transform,
                                          target_transform=target_transform)
        self.imgs = self.samples

- extension란이 `IMG_EXTENSIONS`로 변경되며, samples가 image sample로 변경된 것 밖에 없다.